In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import pyc3dserver as c3d
%matplotlib inline

# del utils
import utils

In [3]:
  
# load data and parameters
participant_folders = utils.get_participants_data_folders()
participant_numbers = utils.load_participant_numbers()
song_genres = utils.load_song_genres()
song_names = utils.load_song_names()
channel_names = utils.load_channel_name_mapping()


In [5]:
itf = c3d.c3dserver()

ind = range(0, len(participant_folders) * len(song_names)) 
tosave = pd.DataFrame(index=ind, columns=["participant", "song", "genre", "cop_x", "cop_y"])
tosave = tosave.astype(object)

i = -1
for i_ptp, participant_folder in enumerate(participant_folders):
    # list all c3d files in directory
    c3d_files = sorted(list(participant_folder.glob('*.c3d')))
    
    for file in c3d_files:
        i = i + 1
        song_name = utils.path_to_song_name(file)
        participant_number = utils.path_to_participant_number(file)
        
        print(file)
        
        # load the c3d file
        ret = c3d.open_c3d(itf, file)
        fp = c3d.get_fp_output(itf)
        
        # get the forceplate data
        df = pd.DataFrame(c3d.get_dict_forces(itf)['DATA'])
        df = df.rename(columns=channel_names)
        
        # close the c3d file
        ret = c3d.close_c3d(itf)

        # specify the start and end frames
        start_time, end_time = utils.get_start_end_frames(song_name, participant_number)
        df = df.loc[start_time:end_time]
        df = df.reset_index(drop=True)

        # calculate the center of pressure
        # formula found here http://silver.neep.wisc.edu/~lakes/BME315L2a.pdf
        cop_x = -1 * df["Moment_y"] / df["Force_z"]
        cop_y = df["Moment_x"] / df["Force_z"]
        
        # store data
        tosave.loc[i, "participant"] = participant_number
        tosave.loc[i, "song"] = song_name
        tosave.loc[i, "genre"] = song_genres[song_name]
        tosave.loc[i, "cop_x"] = [list(cop_x)]
        tosave.loc[i, "cop_y"] = [list(cop_y)]

Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-01-Bloodline-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-02-BackInBlack-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-03-Juju-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-04-SoWhat-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-05-TheOcean-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P1-Filled-In\220127-P1-06-GoIntoTheWater-filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P2-Filled-in\220127-P2-01-TheOcean-Filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_project\data\final_data\P2-Filled-in\220127-P2-02-Juju-Filled.c3d
C:\Users\kendr\Desktop\drummer_mocap_proj

In [9]:
filepath = Path().resolve().parent / "data" / "data-prep_force-plate-center-of-pressure.csv"
tosave.to_csv(filepath, index=False)

In [10]:
tosave

,participant,song,genre,cop_x,cop_y
0,1,Bloodline,Heavy metal,"[[0.1189349889755249, 0.12480694055557251, 0.1...","[[-0.028675459325313568, -0.027246005833148956..."
1,1,BackInBlack,Rock,"[[-0.26532602310180664, -0.259488046169281, -0...","[[0.083261638879776, 0.08744806796312332, 0.06..."
2,1,Juju,Jazz,"[[0.08097628504037857, 0.0813104510307312, 0.0...","[[0.00620765658095479, 0.0026051977183669806, ..."
3,1,SoWhat,Jazz,"[[0.009710974991321564, 0.010020876303315163, ...","[[-0.06650251150131226, -0.06371375918388367, ..."
4,1,TheOcean,Rock,"[[-0.12205449491739273, -0.11852315813302994, ...","[[0.0896940752863884, 0.09128684550523758, 0.0..."
5,1,GoIntoTheWater,Heavy metal,"[[-0.003453562967479229, -0.009510243311524391...","[[-0.0603785514831543, -0.07643565535545349, -..."
6,2,TheOcean,Rock,"[[0.2065892368555069, 0.2070848047733307, 0.18...","[[0.10842198133468628, 0.10743293911218643, 0...."
7,2,Juju,Jazz,"[[0.30011460185050964, 0.30128777027130127, 0....","[[0.06725866347551346, 0.06468961387872696, 0...."
8,2,BackInBlack,Rock,"[[0.15600737929344177, 0.14958202838897705, 0....","[[0.05809362605214119, 0.03769451752305031, 0...."
9,2,Bloodline,Heavy metal,"[[0.2772296667098999, 0.27321162819862366, 0.2...","[[-0.06836538016796112, -0.08467936515808105, ..."
